In [9]:
import datetime
import math
import string

import gspread
import pandas as pd

In [3]:
gc = gspread.service_account(filename="niwako-jupyter-482b54b4ac8a.json")
sh = gc.open_by_key("1bWv49xiZ4stjf8V6AnPWnDSyU9rbHe1R62Fw9eKyK98")

In [7]:
df = pd.DataFrame(sh.sheet1.get_all_records())
df["expiration_date"] = pd.to_datetime(df["expiration_date"]).dt.date
df["is_opened"] = df["is_opened"] == "TRUE"
df["priority"] = (df["expiration_date"] - datetime.date.today()).dt.days - df[
    "servings"
]
df = df.sort_values("priority")
df.head()

,expiration_date,content,storage_location,is_opened,servings,type
0,1/13/2020,わかめとしじみ （味噌汁トッピング）,Pantry,TRUE,30,Meal
1,10/15/2020,乾燥ねぎ,Pantry,TRUE,30,Meal
2,9/26/2021,乾燥ミツバ,Pantry,FALSE,6,Meal
3,3/18/2020,あおさ,Pantry,TRUE,15,Meal
4,2/22/2022,春雨スープ,Pantry,FALSE,10,Meal


In [14]:
dict(zip(df.columns, string.ascii_uppercase))

{'expiration_date': 'A',
 'content': 'B',
 'storage_location': 'C',
 'is_opened': 'D',
 'servings': 'E',
 'type': 'F'}

In [8]:
df.columns

Index(['expiration_date', 'content', 'storage_location', 'is_opened',
       'servings', 'type'],
      dtype='object')

In [4]:
entry = df[
    (df["expiration_date"] > datetime.date.today()) & (df["type"] == "Meal")
].iloc[0]
entry

expiration_date     2021-05-31
content                   ラーメン
storage_location       Freezer
is_opened                False
servings                     2
type                      Meal
priority                    26
Name: 45, dtype: object

In [74]:
days = (entry.expiration_date - datetime.date.today()).days
period = days // math.ceil(entry.servings / 3)

In [22]:
def generate_schedule(entries):
    schedule = {}
    for entry in entries:
        curr = datetime.date.today()
        servings = entry["servings"]
        days = (entry["expiration_date"] - datetime.date.today()).days
        period = days // math.ceil(servings / 3)
        for i, serving in enumerate(range(servings)):
            while curr in schedule:
                curr += datetime.timedelta(days=1)
            if i != 0 and i % 3 == 0:
                curr += datetime.timedelta(days=period)
            if (curr - datetime.date.today()).days > 14:
                break
            schedule[curr] = entry
    return schedule

In [23]:
entries = df[
    (df["expiration_date"] > datetime.date.today()) & (df["type"] == "Meal")
].to_dict(orient="records")

In [24]:
generate_schedule(entries)

{datetime.date(2021, 5, 3): {'expiration_date': datetime.date(2021, 5, 31),
  'content': 'ラーメン',
  'storage_location': 'Freezer',
  'is_opened': False,
  'servings': 2,
  'type': 'Meal',
  'priority': 26},
 datetime.date(2021, 5, 4): {'expiration_date': datetime.date(2021, 5, 31),
  'content': 'ラーメン',
  'storage_location': 'Freezer',
  'is_opened': False,
  'servings': 2,
  'type': 'Meal',
  'priority': 26},
 datetime.date(2021, 5, 5): {'expiration_date': datetime.date(2021, 6, 23),
  'content': '黄金揚げもち',
  'storage_location': 'Pantry',
  'is_opened': False,
  'servings': 14,
  'type': 'Meal',
  'priority': 37},
 datetime.date(2021, 5, 6): {'expiration_date': datetime.date(2021, 6, 23),
  'content': '黄金揚げもち',
  'storage_location': 'Pantry',
  'is_opened': False,
  'servings': 14,
  'type': 'Meal',
  'priority': 37},
 datetime.date(2021, 5, 7): {'expiration_date': datetime.date(2021, 6, 23),
  'content': '黄金揚げもち',
  'storage_location': 'Pantry',
  'is_opened': False,
  'servings': 14,
 

In [6]:
df

,expiration_date,content,storage_location,is_opened,servings,type,priority
7,2019-06-19,乾燥ひじき,Pantry,False,10,Meal,-694
6,2019-11-01,乾燥ひじき,Pantry,True,4,Meal,-553
0,2020-01-13,わかめとしじみ （味噌汁トッピング）,Pantry,True,30,Meal,-506
9,2020-02-19,七味唐辛子,Pantry,False,28,Meal,-467
3,2020-03-18,あおさ,Pantry,True,15,Meal,-426
26,2020-09-01,梅干しタブレット,Pantry,True,18,Snack,-262
5,2020-09-10,鮭胡麻ふりかけ,Pantry,False,6,Meal,-241
1,2020-10-15,乾燥ねぎ,Pantry,True,30,Meal,-230
29,2021-03-10,Yokumoku cookies,Fridge,False,4,Sweets,-58
10,2021-04-13,キンショウバイ,Freezer,True,30,Meal,-50
